In [2]:
import os
import csv
import psycopg2
import mysql
import MySQLdb
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, DATE, TIMESTAMP, VARCHAR, FLOAT, INTEGER
from psycopg2 import sql
import pandas as pd
from extras import create_table_leanx as ctl
from extras import table_schemas

/Users/joannamasin-buck/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# ctl.get_raw_query_as_str(table_name='VBAK')

In [3]:
db_host = 'academymysql.cluster-ckynnbglhixw.eu-central-1.rds.amazonaws.com'
db_port = '3306'
db_name = 'ocpm_inv_test'
db_user = 'ocpm_inv_test'
db_password = 'Celonis1234!'

# Construct the connection string
connection_string = sql.SQL("dbname={} user={} password={} host={} port={}").format(
    sql.Identifier(db_name),
    sql.Identifier(db_user),
    sql.Identifier(db_password),
    sql.Identifier(db_host),
    sql.Identifier(db_port)
)

In [ ]:
# connection = psycopg2.connect(f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}")
# cursor = connection.cursor()

# cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'ocpm_om';")

# cursor.execute("SELECT schema_name FROM information_schema.schemata;")
# cursor.execute("""
# SELECT table_name
# FROM information_schema.tables
# WHERE table_type = 'BASE TABLE' AND table_schema = 'ocpm_om';
# """)
# table_names = cursor.fetchall()
# print("Tables in the database:")
# for table in table_names:
#     print(table[0])

# cursor.close()
# connection.close()

In [ ]:
#check timestamps / dates in schemas
# ts = table_schemas.table_schemas

# def checktimecolumns(table_name: str):
#     list_of_columns = []
#     for column in ctl.required_table_columns[table_name]:
#         if ts[table_name][column] == DATE or ts[table_name][column] == TIMESTAMP:
#             list_of_columns.append(column)
#     return list_of_columns 

# checktimecolumns('VBFA')

In [4]:
connection = MySQLdb.connect(db=db_name, user=db_user, passwd=db_password, host=db_host)
cursor = connection.cursor()
# root_folder = "data/OCPM 011124 1858"
root_folder = "data/inv/ocpm"

ts = table_schemas.table_schemas
ts_csv = table_schemas.columns_to_parse

def find_csv_files(root_folder):
    csv_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

def create_or_overwrite_table(csv_file_path, cursor):
    table_name = csv_file_path.split('\\')[-1]
    table_name = table_name.split('.')[0]
    print(f"Creating or overwriting table: {table_name} with data from: {csv_file_path}")

    df = pd.read_csv(csv_file_path, parse_dates=ts_csv[table_name])
    engine = create_engine(f"mysql+mysqldb://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, dtype=ts[table_name])

csv_files = find_csv_files(root_folder)
for csv_file in csv_files:
    create_or_overwrite_table(csv_file, cursor)

cursor.close()
connection.close()

OperationalError: (2003, "Can't connect to MySQL server on 'academymysql.cluster-ckynnbglhixw.eu-central-1.rds.amazonaws.com:3306' (60)")